In [1]:
import sqlite3
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
L = "https://books.toscrape.com/index.html"

In [3]:
def Link_Of_Page(LINK):
    response = requests.get(LINK)
    soup = BeautifulSoup(response.text,"html.parser")
    catalogue = soup.find_all('a')
    link=[]
    for c in catalogue:
        link.append(c.attrs['href'])
    link = link[3:53]
    return link

In [4]:
link=Link_Of_Page(L)
rates = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}

In [5]:
def Details_Category(i):
    url=f"https://books.toscrape.com/{link[i]}"
    response=requests.get(url)
    soup=BeautifulSoup(response.text,"html.parser")
    art =soup.find_all('article',attrs={"class":"product_pod"})
    Name=soup.find('ul',attrs={'class':'nav nav-list'}).find_all('li')[i+1].find('a').get_text().strip()
    print(f'Name Of Category: {Name}')
    print(f'Link Of Category: {url}')
    print('*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*')
    return art,soup,Name,url

In [6]:
def Details_Book(link_):
    response=requests.get(link_)
    SOUP=BeautifulSoup(response.text,"html.parser")
    quantity=SOUP.find('table',attrs={'class':'table table-striped'}).find_all('tr')[5].find('td').get_text()
    quantity=quantity.split()
    quantity=quantity[2][1:]
    return quantity

In [7]:
def Details_Page(j):
    
    name =j.find('h3').find('a')['title']
    price=float(j.find('p',attrs={"class":"price_color"}).get_text().split('£')[1])
    rate =j.find('p').get('class')[1]
    l=j.find('h3').find('a').get('href')
    l=l.split('../')
    l=''.join(l)
    link_ =f"https://books.toscrape.com/catalogue/{l}"
    quantity=Details_Book(link_)
    print(f'Book name: {name}\nPrice: {price}\nRate: {rates[rate]}\nBook link: {link_}\nquantity: {quantity}\n')
    print('-----------------------------------------------')
    return name,price,rate,link_,quantity

In [8]:
def Details_Inner_Page(i,soup):
    next_page= soup.find('li',attrs={'class':'next'}).find('a').attrs['href']
    link[i]=link[i].split('/')
    link[i][-1]=next_page
    link[i]='/'.join(link[i]) 

In [9]:
def Get_Books(L):
    for i in range(len(link)):
        art,soup,Name,url = Details_Category(i)
        
        connection = sqlite3.connect("Books.db")
        row = connection.execute("INSERT INTO Category (Name) VALUES (?)", [Name])
        lr=row.lastrowid
        connection.commit()
        connection.close()
        
        for j in art:
            name,price,rate,link_,quantity = Details_Page(j)
            
            connection = sqlite3.connect("Books.db")
            row = connection.execute("INSERT INTO Book (Name,Price,Rate,Quantity,id_c) VALUES (?,?,?,?,?)", [name,price,rate,quantity,lr])
            connection.commit()
            connection.close()
        
        try:
            while(True):
                pages=soup.find('li',attrs={'class':'current'}).get_text().strip()
                if(int(pages[5:7])<=int(pages[-2:])):
                    Details_Inner_Page(i,soup)
                    art,soup,Name,url = Details_Category(i)
                    for j in art:
                        name,price,rate,link_,quantity = Details_Page(j)
                        
                        connection = sqlite3.connect("Books.db")
                        row = connection.execute("INSERT INTO Book (Name,Price,Rate,Quantity,id_c) VALUES (?,?,?,?,?)", [name,price,rate,quantity,lr])
                        connection.commit()
                        connection.close()
            
                else:
                    break 
        except:
            continue

In [10]:
Get_Books(L)

Name Of Category: Travel
Link Of Category: https://books.toscrape.com/catalogue/category/books/travel_2/index.html
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Book name: It's Only the Himalayas
Price: 45.17
Rate: 2
Book link: https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html
quantity: 19

-----------------------------------------------
Book name: Full Moon over Noahâs Ark: An Odyssey to Mount Ararat and Beyond
Price: 49.43
Rate: 4
Book link: https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html
quantity: 15

-----------------------------------------------
Book name: See America: A Celebration of Our National Parks & Treasured Sites
Price: 48.87
Rate: 3
Book link: https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html
quantity: 14

-----------------------------------------------
Book name: Vagabonding: An Uncommon 

Book name: The Exiled
Price: 43.45
Rate: 3
Book link: https://books.toscrape.com/catalogue/the-exiled_247/index.html
quantity: 3

-----------------------------------------------
Book name: The Cuckoo's Calling (Cormoran Strike #1)
Price: 19.21
Rate: 1
Book link: https://books.toscrape.com/catalogue/the-cuckoos-calling-cormoran-strike-1_239/index.html
quantity: 3

-----------------------------------------------
Book name: Extreme Prey (Lucas Davenport #26)
Price: 25.4
Rate: 3
Book link: https://books.toscrape.com/catalogue/extreme-prey-lucas-davenport-26_154/index.html
quantity: 3

-----------------------------------------------
Book name: Career of Evil (Cormoran Strike #3)
Price: 24.72
Rate: 2
Book link: https://books.toscrape.com/catalogue/career-of-evil-cormoran-strike-3_137/index.html
quantity: 3

-----------------------------------------------
Book name: The No. 1 Ladies' Detective Agency (No. 1 Ladies' Detective Agency #1)
Price: 57.7
Rate: 4
Book link: https://books.toscrape.com

Book name: Tsubasa: WoRLD CHRoNiCLE 2 (Tsubasa WoRLD CHRoNiCLE #2)
Price: 16.28
Rate: 1
Book link: https://books.toscrape.com/catalogue/tsubasa-world-chronicle-2-tsubasa-world-chronicle-2_949/index.html
quantity: 16

-----------------------------------------------
Book name: This One Summer
Price: 19.49
Rate: 4
Book link: https://books.toscrape.com/catalogue/this-one-summer_947/index.html
quantity: 16

-----------------------------------------------
Book name: The Nameless City (The Nameless City #1)
Price: 38.16
Rate: 4
Book link: https://books.toscrape.com/catalogue/the-nameless-city-the-nameless-city-1_940/index.html
quantity: 16

-----------------------------------------------
Book name: Saga, Volume 5 (Saga (Collected Editions) #5)
Price: 51.04
Rate: 2
Book link: https://books.toscrape.com/catalogue/saga-volume-5-saga-collected-editions-5_923/index.html
quantity: 16

-----------------------------------------------
Book name: Rat Queens, Vol. 3: Demons (Rat Queens (Collected Editio

Book name: El Deafo
Price: 57.62
Rate: 5
Book link: https://books.toscrape.com/catalogue/el-deafo_691/index.html
quantity: 14

-----------------------------------------------
Book name: Batman: Europa
Price: 32.01
Rate: 2
Book link: https://books.toscrape.com/catalogue/batman-europa_668/index.html
quantity: 14

-----------------------------------------------
Book name: Art Ops Vol. 1
Price: 48.8
Rate: 3
Book link: https://books.toscrape.com/catalogue/art-ops-vol-1_664/index.html
quantity: 14

-----------------------------------------------
Book name: Adulthood Is a Myth: A "Sarah's Scribbles" Collection
Price: 10.9
Rate: 2
Book link: https://books.toscrape.com/catalogue/adulthood-is-a-myth-a-sarahs-scribbles-collection_659/index.html
quantity: 14

-----------------------------------------------
Book name: Fruits Basket, Vol. 9 (Fruits Basket #9)
Price: 33.95
Rate: 4
Book link: https://books.toscrape.com/catalogue/fruits-basket-vol-9-fruits-basket-9_563/index.html
quantity: 9

---------

Book name: Paper Girls, Vol. 1 (Paper Girls #1-5)
Price: 21.71
Rate: 4
Book link: https://books.toscrape.com/catalogue/paper-girls-vol-1-paper-girls-1-5_44/index.html
quantity: 1

-----------------------------------------------
Book name: Ouran High School Host Club, Vol. 1 (Ouran High School Host Club #1)
Price: 29.87
Rate: 3
Book link: https://books.toscrape.com/catalogue/ouran-high-school-host-club-vol-1-ouran-high-school-host-club-1_43/index.html
quantity: 1

-----------------------------------------------
Book name: Ms. Marvel, Vol. 1: No Normal (Ms. Marvel (2014-2015) #1)
Price: 39.39
Rate: 4
Book link: https://books.toscrape.com/catalogue/ms-marvel-vol-1-no-normal-ms-marvel-2014-2015-1_34/index.html
quantity: 1

-----------------------------------------------
Book name: Hawkeye, Vol. 1: My Life as a Weapon (Hawkeye #1)
Price: 45.24
Rate: 3
Book link: https://books.toscrape.com/catalogue/hawkeye-vol-1-my-life-as-a-weapon-hawkeye-1_24/index.html
quantity: 1

----------------------

...

Name Of Category: Parenting
Link Of Category: https://books.toscrape.com/catalogue/category/books/parenting_28/index.html
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Book name: Catastrophic Happiness: Finding Joy in Childhood's Messy Years
Price: 37.35
Rate: 2
Book link: https://books.toscrape.com/catalogue/catastrophic-happiness-finding-joy-in-childhoods-messy-years_138/index.html
quantity: 3

-----------------------------------------------
Name Of Category: Adult Fiction
Link Of Category: https://books.toscrape.com/catalogue/category/books/adult-fiction_29/index.html
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Book name: Fifty Shades Freed (Fifty Shades #3)
Price: 15.36
Rate: 5
Book link: https://books.toscrape.com/catalogue/fifty-shades-freed-fifty-shades-3_156/index.html
quantity: 3

-----------------------------------------------
Name Of Category: Humor
Link Of Category: https://books.toscrape.com/catalogue/category/books/humor_3

Book name: Thomas Jefferson and the Tripoli Pirates: The Forgotten War That Changed American History
Price: 59.64
Rate: 1
Book link: https://books.toscrape.com/catalogue/thomas-jefferson-and-the-tripoli-pirates-the-forgotten-war-that-changed-american-history_867/index.html
quantity: 15

-----------------------------------------------
Book name: Zealot: The Life and Times of Jesus of Nazareth
Price: 24.7
Rate: 3
Book link: https://books.toscrape.com/catalogue/zealot-the-life-and-times-of-jesus-of-nazareth_785/index.html
quantity: 14

-----------------------------------------------
Book name: A Distant Mirror: The Calamitous 14th Century
Price: 14.58
Rate: 3
Book link: https://books.toscrape.com/catalogue/a-distant-mirror-the-calamitous-14th-century_652/index.html
quantity: 14

-----------------------------------------------
Book name: 1491: New Revelations of the Americas Before Columbus
Price: 21.8
Rate: 3
Book link: https://books.toscrape.com/catalogue/1491-new-revelations-of-the-amer

Book name: Everyday Italian: 125 Simple and Delicious Recipes
Price: 20.1
Rate: 5
Book link: https://books.toscrape.com/catalogue/everyday-italian-125-simple-and-delicious-recipes_618/index.html
quantity: 12

-----------------------------------------------
Book name: A la Mode: 120 Recipes in 60 Pairings: Pies, Tarts, Cakes, Crisps, and More Topped with Ice Cream, Gelato, Frozen Custard, and More
Price: 38.77
Rate: 1
Book link: https://books.toscrape.com/catalogue/a-la-mode-120-recipes-in-60-pairings-pies-tarts-cakes-crisps-and-more-topped-with-ice-cream-gelato-frozen-custard-and-more_611/index.html
quantity: 12

-----------------------------------------------
Book name: Cravings: Recipes for What You Want to Eat
Price: 20.5
Rate: 3
Book link: https://books.toscrape.com/catalogue/cravings-recipes-for-what-you-want-to-eat_589/index.html
quantity: 11

-----------------------------------------------
Book name: The Moosewood Cookbook: Recipes from Moosewood Restaurant, Ithaca, New York
Pri

Book name: Born for This: How to Find the Work You Were Meant to Do
Price: 21.59
Rate: 5
Book link: https://books.toscrape.com/catalogue/born-for-this-how-to-find-the-work-you-were-meant-to-do_588/index.html
quantity: 11

-----------------------------------------------
Book name: The E-Myth Revisited: Why Most Small Businesses Don't Work and What to Do About It
Price: 36.91
Rate: 1
Book link: https://books.toscrape.com/catalogue/the-e-myth-revisited-why-most-small-businesses-dont-work-and-what-to-do-about-it_545/index.html
quantity: 8

-----------------------------------------------
Book name: Rich Dad, Poor Dad
Price: 51.74
Rate: 1
Book link: https://books.toscrape.com/catalogue/rich-dad-poor-dad_483/index.html
quantity: 7

-----------------------------------------------
Book name: The Lean Startup: How Today's Entrepreneurs Use Continuous Innovation to Create Radically Successful Businesses
Price: 33.92
Rate: 3
Book link: https://books.toscrape.com/catalogue/the-lean-startup-how-toda

Book name: Logan Kade (Fallen Crest High #5.5)
Price: 13.12
Rate: 2
Book link: https://books.toscrape.com/catalogue/logan-kade-fallen-crest-high-55_384/index.html
quantity: 5

-----------------------------------------------
Name Of Category: Self Help
Link Of Category: https://books.toscrape.com/catalogue/category/books/self-help_41/index.html
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Book name: Online Marketing for Busy Authors: A Step-By-Step Guide
Price: 46.35
Rate: 1
Book link: https://books.toscrape.com/catalogue/online-marketing-for-busy-authors-a-step-by-step-guide_913/index.html
quantity: 16

-----------------------------------------------
Book name: How to Be Miserable: 40 Strategies You Already Use
Price: 46.03
Rate: 1
Book link: https://books.toscrape.com/catalogue/how-to-be-miserable-40-strategies-you-already-use_897/index.html
quantity: 16

-----------------------------------------------
Book name: Overload: How to Unplug, Unwind, and Unleash Yo